In [169]:
import networkx as nx
import pandas as pd
from pyvis.network import Network
import community as community_louvain

In [170]:
import os
os.chdir(r"X:\My Projects\SNA\facebook-sna-analysis")

In [171]:
file_path = "data/raw/facebook_combined.txt.gz"

G = nx.read_edgelist(file_path, nodetype=int)
print("Number of nodes (actors):", G.number_of_nodes())
print("Number of edges (connections):", G.number_of_edges())
print("Is the graph connected:", nx.is_connected(G))
print("Number of connected components:", nx.number_connected_components(G))

Number of nodes (actors): 4039
Number of edges (connections): 88234
Is the graph connected: True
Number of connected components: 1


In [172]:
largest_cc = max(nx.connected_components(G), key=len)
G_cc = G.subgraph(largest_cc)

partition = community_louvain.best_partition(G_cc)

In [173]:
centrality_df=pd.read_csv(r'outputs\tables\facebook_centrality_metrics.csv')

In [174]:
centrality_df["community"] = centrality_df["node"].map(partition)

In [175]:
top_nodes = (
    centrality_df
    .sort_values("betweenness", ascending=False)
    .head(300)["node"]
    .tolist()
)

subG = G_cc.subgraph(top_nodes)

In [176]:
net = Network(
    height="750px",
    width="100%",
    bgcolor="#ffffff",
    font_color="black",
    notebook=True
)

# enable physics controls
net.toggle_physics(True)

In [177]:
# Add Nodes (with community + size)
for node in subG.nodes():
    net.add_node(
        node,
        label=str(node),
        title=(
            f"Node: {node} "
            f"Community: {partition[node]} "
            f"Degree: {G.degree(node)} "
            f"Betweenness: "
            f"{centrality_df.loc[centrality_df['node']==node,'betweenness'].values[0]:.6f}"
        ),
        value=centrality_df.loc[
            centrality_df["node"] == node, "betweenness"
        ].values[0] * 5000,
        group=partition[node]
    )

# Add edges
for u, v in subG.edges():
    net.add_edge(u, v)


In [179]:
net.set_options("""
var options = {
  "interaction": {
    "hover": true,
    "tooltipDelay": 100,
    "hideEdgesOnDrag": true,
    "hideNodesOnDrag": false,
    "navigationButtons": true,
    "keyboard": true
  },
  "physics": {
    "forceAtlas2Based": {
      "gravitationalConstant": -50,
      "centralGravity": 0.01,
      "springLength": 100,
      "springConstant": 0.08
    },
    "solver": "forceAtlas2Based",
    "stabilization": {
      "iterations": 200,
      "fit": true
    }
  }
}
""")

In [180]:
net.show("facebook_communities_dynamic.html")

facebook_communities_dynamic.html
